In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Segment, Range1d
from bokeh.layouts import layout
from bokeh.io import push_notebook, show, output_notebook
import numpy as np
from tensorflow.keras.models import load_model
import sys
import pandas as pd
import paho.mqtt.client as mqtt
import json
import datetime
import time
from math import *
import random
import _thread
import yaml



# 更新函數，每次呼叫時更新數據
def update(message):
    global x, y_data, grap_id, rawData, record_count, predict_start
    
    # 解碼 MQTT 數據
    if test_mode:
        data = message
    else:
        payload = message.payload.decode()
        data = json.loads(payload)

    
    mqData = np.array(data)
    rawData.append(mqData)

    # times = mqData[:, 0]
    channel1 = mqData[:, 1]
    channel2 = mqData[:, 2]
    channel3 = mqData[:, 3]
    channel4 = mqData[:, 4]
    channel5 = mqData[:, 5]
    channel6 = mqData[:, 6]
    channel7 = mqData[:, 7]
    channel8 = mqData[:, 8]
    channel9 = mqData[:, 9]
    channel10 = mqData[:, 10]
    channel11 = mqData[:, 11]
    channel12 = mqData[:, 12]




    
    ### 使用模型进行预测 ###

    # 使用測試隨機數
    # ranges = np.array([[1.7, 2], [6.7, 7], [-7, -6.6], [-4, 2], [-2, 2], [-8, -3], [0, 0.3], [9, 9.3], [-4.3, -4], [0, 5], [-2, 2], [-6, -4]])
    # 使用 numpy.random.uniform 生成具有不同範圍的數據
    # dataForTest = np.column_stack([np.random.uniform(low=low, high=high, size=30) for low, high in ranges])
    # print('-----------------')
    # dataForTest = dataForTest[np.newaxis, ...]
    # print('transforming test random data shape:', dataForTest.shape)
    # print('data for test:', dataForTest)


    # 使用訓練數據展示結果
    # dataForTrain = trained_data_x[record_count * data_length : record_count * data_length + data_length, :]
    # dataForTrain = dataForTrain[np.newaxis, ...]
    # print('transforming train data shape:', dataForTrain.shape)
    # print('data for train:', dataForTrain)

    if (record_count + 1) * data_length >= (predict_start + 1) * window_size + window_size:
    # if record_count > 1:
        # 有數據量滿足最小2倍窗口才能開始推理
        npRawData = np.concatenate(rawData, axis=0)
        fifoData = np.empty((window_size, window_size, 12), dtype=float)
        for i in range(window_size):
            tmp = npRawData[predict_start * window_size + i : predict_start * window_size + window_size + i, 1:]
            # print('tmp data shape:', tmp.shape)
            # print('tmp data:', tmp)
            if is_normalize:
                fifoData[:, i, :] = tmp
            else:
                fifoData[:, i, :] = tmp
            # print('fifo:', fifoData)


        # print('transforming fifo data shape:', fifoData.shape)
        # print('fifo:', fifoData)
        # print('np raw data shape:', npRawData.shape)
        # print(predict_start * window_size + window_size)

        y_pred = model.predict(fifoData)
        # predictions.append(y_pred)

        # 将预测结果转换为 NumPy 数组
        # predictionsNp = np.array(predictions)
        predictionsNp = np.array(y_pred)
        predictionsNp = predictionsNp.flatten()

        # 判斷是否過動
        hyperActionCount = np.sum(predictionsNp >= 0.5)
        if hyperActionCount >= 50:
            publish()
        # print('---------')
        # print('y_pred:', y_pred)
        # print('y_pred shape:', y_pred.shape)
        # print('------------------')
        # print('predictoin NP:', predictionsNp)
        # print('predictions shape:', predictionsNp.shape)
        # 提取预测类别的最大概率 >0.5 => 1, <0.5 => 0
        # predicted_classes = predictionsNp[-1]
        # 只會有一個結果, 值為 0 或者 1
        # print(predicted_classes[0])
        # prediction_label = round(predicted_classes[0])
        # print('predicted_classes', predicted_classes)
        # print('prediction_label', prediction_label)
        # print('++++++++++++++++')
        # print(predict_start * window_size, predict_start * window_size + window_size)
        # print('++++++++++++++++')
        predict_start += 1
        source13.stream({'x': np.arange(predict_start * window_size, predict_start * window_size + window_size), 'y': predictionsNp})
        separation.stream({'x': np.arange(predict_start * window_size, predict_start * window_size + window_size), 'y': np.full(window_size, 0.5)})




    # 更新數據源的數據
    grap_id += data_length

    record_count = record_count + 1


    # 更新數據源的數據
    grap_id += data_length
    source1.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel1})
    source2.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel2})
    source3.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel3})
    source4.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel4})
    source5.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel5})
    source6.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel6})
    source7.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel7})
    source8.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel8})
    source9.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel9})
    source10.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel10})
    source11.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel11})
    source12.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel12})

    # print('predictions count', predict_start)
    # print('update count', record_count)


    if export_csv and record_count % 1000 == 0:
        exportCSV()

    push_notebook()

    

def exportCSV():
    global rawData
    csvData = np.concatenate(rawData, axis=0)
    timestamps = csvData[:, 0]
    ax1 = csvData[:, 1]
    ay1 = csvData[:, 2]
    az1 = csvData[:, 3]
    gx1 = csvData[:, 4]
    gy1 = csvData[:, 5]
    gz1 = csvData[:, 6]

    ax2 = csvData[:, 7] 
    ay2 = csvData[:, 8]
    az2 = csvData[:, 9]
    gx2 = csvData[:, 10]
    gy2 = csvData[:, 11]
    gz2 = csvData[:, 12]

    # 将时间戳转换为本地时间元组
    # new_times = np.vectorize(transformTimestamps)(timestamps)

    # dictionary of lists
    dict = {'Time': timestamps, 'Ax1': ax1, 'Ay1': ay1, 'Az1': az1, 'Gx1': gx1, 'Gy1': gy1, 'Gz1': gz1, 'Ax2': ax2, 'Ay2': ay2, 'Az2': az2, 'Gx2': gx2, 'Gy2': gy2, 'Gz2': gz2}
        
    df = pd.DataFrame(dict)
    filename = 'output.csv'
    df.to_csv(filename, index=False)


def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print("Failed to connect, return code %d\n", rc)

def on_message(client, userdata, message):
    print("Received data from esp32 via MQTT")
    update(message)
    # print("Received message on topic %s: %s" % (message.topic, message.payload.decode()))

def publish():
    topic = 'Command'
    message = 'vibrator'
    result = client.publish(topic, message)
    status = result[0]
    # if status == 0:
    #     print('Command publish successfully')
    # else:
    #     print('Command publish failed')

def connect_mqtt_server():
    # 连接MQTT代理
    # broker_address = "broker.emqx.io"
    broker_address = config['mqtt']['broker_address']
    # broker_address = "broker.mqttdashboard.com"
    # broker_address = "test.mosquitto.org"
    port = config['mqtt']['port']
    username = config['mqtt']['username']
    password = config['mqtt']['password']

    client = mqtt.Client()
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.on_message = on_message

    client.connect(broker_address, port)

    # 订阅消息
    client.subscribe(topic)

    _thread.start_new_thread(client.loop_forever, (1,))

    return client


def normalize(X):    
    """    对矩阵X进行0-1标准化    """    
    X_min = np.min(X, axis=0)    
    X_max = np.max(X, axis=0)    
    X_norm = (X - X_min) / (X_max - X_min)    
    return X_norm


def z_score_normalize(data):    
    mean = np.mean(data, axis=0)    
    std_dev = np.std(data, axis=0)    
    normalized_data = (data - mean) / std_dev    
    return normalized_data



################################ Main Program ################################

# 初始化數據/設定
with open('./config.yaml', 'r') as f:
    config = yaml.safe_load(f)


data_length = config['data']['length']
data_channels = config['data']['channels']
window_size = config['data']['window_size']
trained_data_path = config['data']['train_data_path']
topic = config['mqtt']['topic']
export_csv = config['env']['export_csv']
is_normalize = config['data']['normalize']


x = np.arange(data_length)

y_data = [np.zeros(data_length) for _ in range(data_channels + 1)]

grap_id = 0

record_count = 0

rawData = []

predict_start = 0

predict_x = np.arange(window_size)
predict_y = np.zeros(window_size)





trained_data = pd.read_csv(trained_data_path + 'a1_1.csv')
trained_data_x = trained_data.iloc[:, 1:-1].values


########### LOAD MODEL ###############
predictions = []
model_path = config['model']['pb_path']
model = load_model(model_path, compile=False)
model.compile()
########### LOAD MODEL ###############



source1 = ColumnDataSource(data={'x': x, 'y': y_data[1]})
source2 = ColumnDataSource(data={'x': x, 'y': y_data[2]})
source3 = ColumnDataSource(data={'x': x, 'y': y_data[3]})
source4 = ColumnDataSource(data={'x': x, 'y': y_data[4]})
source5 = ColumnDataSource(data={'x': x, 'y': y_data[5]})
source6 = ColumnDataSource(data={'x': x, 'y': y_data[6]})

source7 = ColumnDataSource(data={'x': x, 'y': y_data[7]})
source8 = ColumnDataSource(data={'x': x, 'y': y_data[8]})
source9 = ColumnDataSource(data={'x': x, 'y': y_data[9]})
source10 = ColumnDataSource(data={'x': x, 'y': y_data[10]})
source11 = ColumnDataSource(data={'x': x, 'y': y_data[11]})
source12 = ColumnDataSource(data={'x': x, 'y': y_data[12]})
source13 = ColumnDataSource(data={'x': [], 'y': []})
separation = ColumnDataSource(data={'x': [], 'y': []})



# 創建 Bokeh 的線型圖
plots = []
plot1 = figure(height=150, title="MPU1", tools="crosshair,pan,reset,save,wheel_zoom")
line_ax1 = plot1.line('x', 'y', source=source1, line_width=2, line_alpha=0.8, line_color='black', legend_label='AX1')
line_ay1 = plot1.line('x', 'y', source=source2, line_width=2, line_alpha=0.8, line_color='blue', legend_label='AY1')
line_az1 = plot1.line('x', 'y', source=source3, line_width=2, line_alpha=0.8, line_color='red', legend_label='AZ1')
line_gx1 = plot1.line('x', 'y', source=source4, line_width=2, line_alpha=0.8, line_color='green', legend_label='GX1')
line_gy1 = plot1.line('x', 'y', source=source5, line_width=2, line_alpha=0.8, line_color='yellow', legend_label='GY1')
line_gz1 = plot1.line('x', 'y', source=source6, line_width=2, line_alpha=0.8, line_color='pink', legend_label='GZ1')



# plot2 = figure(plot_height=300, title="Channel AY1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot2.line('x', 'y', source=source2, line_width=2, line_alpha=0.8)

# plot3 = figure(plot_height=300, title="Channel AZ1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot3.line('x', 'y', source=source3, line_width=2, line_alpha=0.8)

# plot4 = figure(plot_height=300, title="Channel GX1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot4.line('x', 'y', source=source4, line_width=2, line_alpha=0.8)

# plot5 = figure(plot_height=300, title="Channel GY1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot5.line('x', 'y', source=source5, line_width=2, line_alpha=0.8)

# plot6 = figure(plot_height=300, title="Channel GZ1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot6.line('x', 'y', source=source6, line_width=2, line_alpha=0.8)

plot7 = figure(height=150, title="MPU2", tools="crosshair,pan,reset,save,wheel_zoom")
line_ax2 = plot7.line('x', 'y', source=source7, line_width=2, line_alpha=0.8, line_color='black', legend_label='AX2')
line_ay2 = plot7.line('x', 'y', source=source8, line_width=2, line_alpha=0.8, line_color='blue', legend_label='AY2')
line_az2 = plot7.line('x', 'y', source=source9, line_width=2, line_alpha=0.8, line_color='red', legend_label='AZ2')
line_gx2 = plot7.line('x', 'y', source=source10, line_width=2, line_alpha=0.8, line_color='green', legend_label='GX2')
line_gy2 = plot7.line('x', 'y', source=source11, line_width=2, line_alpha=0.8, line_color='yellow', legend_label='GY2')
line_gz2 = plot7.line('x', 'y', source=source12, line_width=2, line_alpha=0.8, line_color='pink', legend_label='GZ2')


# plot8 = figure(plot_height=300, title="Channel AY2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot8.line('x', 'y', source=source8, line_width=2, line_alpha=0.8)

# plot9 = figure(plot_height=300, title="Channel AZ2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot9.line('x', 'y', source=source9, line_width=2, line_alpha=0.8)

# plot10 = figure(plot_height=300, title="Channel GX2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot10.line('x', 'y', source=source10, line_width=2, line_alpha=0.8)

# plot11 = figure(plot_height=300, title="Channel GY2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot11.line('x', 'y', source=source11, line_width=2, line_alpha=0.8)

# plot12 = figure(plot_height=300, title="Channel GZ2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot12.line('x', 'y', source=source12, line_width=2, line_alpha=0.8)

plot13 = figure(height=150, title="Predictions", tools="crosshair,pan,reset,save,wheel_zoom")
line = plot13.line('x', 'y', source=separation, line_width=2, line_alpha=0.8, line_color='blue')
plot13.add_glyph(Segment(x0=0, y0=0.5, x1=1, y1=0.5, line_color='blue', line_width=2))
plot13.y_range = Range1d(0, 1)
line = plot13.line('x', 'y', source=source13, line_width=2, line_alpha=0.8, line_color='orange')


plots.append(plot1)
plots.append(plot7)
plots.append(plot13)
# plots.append(plot3)
# plots.append(plot4)
# plots.append(plot5)
# plots.append(plot6)
# plots.append(plot7)
# plots.append(plot8)
# plots.append(plot9)
# plots.append(plot10)
# plots.append(plot11)
# plots.append(plot12)


grid = layout(plots, sizing_mode='scale_width')

output_notebook()

show(grid, notebook_handle=True)


test_mode = config['env']['test']
if not test_mode:
    client = connect_mqtt_server()

while test_mode:
    test_data = []
    for i in range(data_length):
        test_data.append([
            '('+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + ')', 
            random.uniform(1.7, 2),
            random.uniform(6.7, 7),
            random.uniform(-7, -6.6),
            random.uniform(-4, 2),
            random.uniform(-2, 2),
            random.uniform(-8, -3),
            random.uniform(0, 0.3),
            random.uniform(9, 9.3),
            random.uniform(-4.3, -4),
            random.uniform(0, 5),
            random.uniform(-2, 2),
            random.uniform(-6, -4),
        ])
    update(test_data)
    time.sleep(0.5)



Loading BokehJS ...

Connected to MQTT broker
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 3ms/step
Command publish successfully
4/4 [==============================] - 0s 3ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 6ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 3ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==============================] - 0s 4ms/step
Command publish successfully
4/4 [==========